# 附录：方差分解的计算对比

1. 和R语言
2. Stata语言
3. python内部MA和脉冲响应对比

In [2]:
# 安装并加载必要的包
#install.packages("vars")
library(vars)

# 示例数据
data <- as.data.frame(matrix(rnorm(1000), ncol=5))
colnames(data) <- c("Asset1", "Asset2", "Asset3", "Asset4", "Asset5")

# 选择滞后阶数
lag_selection <- VARselect(data, lag.max=10, type="const")
p <- lag_selection$selection["AIC(n)"]

# 建立VAR模型
var_model <- VAR(data, p=p, type="const")



In [4]:

# 自定义非正交化预测误差方差分解
gfevd_custom <- function(var_model, n.ahead) {
  Phi <- fevd(var_model, n.ahead=n.ahead)$Phi
  Sigma <- cov(residuals(var_model))
  n_vars <- dim(Phi)[1]
  
  # 初始化FEVD矩阵
  fevd_matrix <- array(0, dim=c(n_vars, n_vars, n.ahead))
  
  for (i in 1:n.ahead) {
    theta <- Phi[,,i]
    for (j in 1:n_vars) {
      fevd_matrix[, j, i] <- diag(theta %*% Sigma %*% t(theta)) / sum(diag(theta %*% Sigma %*% t(theta)))
    }
  }
  
  return(fevd_matrix)
}


In [5]:
# 计算GFEVD
gfevd_result <- gfevd_custom(var_model, n.ahead=10)

# 查看结果
print(gfevd_result)

ERROR: Error in 1:n_vars: argument of length 0
